In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from utils.data_loader import LibriSpeechDataLoader
from utils.feature_extractor import FeatureExtractor
import config

# Load character mappings
char_to_num = np.load('char_to_num.npy', allow_pickle=True).item()
num_to_char = np.load('num_to_char.npy', allow_pickle=True).item()

print(f"Loaded character mapping with {len(char_to_num)} characters")

# Initialize data loader and feature extractor
data_loader = LibriSpeechDataLoader(config.DATASET_CONFIG)
data_loader.char_to_num = char_to_num
data_loader.num_to_char = num_to_char

feature_extractor = FeatureExtractor(
    sampling_rate=config.DATASET_CONFIG['sampling_rate'],
    n_mfcc=config.FEATURE_CONFIG['n_mfcc'],
    n_mels=config.FEATURE_CONFIG['n_mels'],
    n_fft=config.FEATURE_CONFIG['n_fft'],
    hop_length=config.FEATURE_CONFIG['hop_length']
)

# Load a sample for feature visualization
print("Loading sample data for feature visualization...")
datasets = data_loader.load_dataset(splits=['train'])

for example in datasets['train'].take(1):
    sample_audio = example['audio'].numpy()
    sample_text = example['text'].numpy().decode('utf-8')
    break

print(f"Sample audio shape: {sample_audio.shape}")
print(f"Sample text: '{sample_text}'")

# Extract and visualize all feature types
print("\nExtracting and visualizing features...")
features = feature_extractor.visualize_features(
    sample_audio, 
    title=f"Feature Extraction - '{sample_text}'"
)

# Print feature shapes
for name, feature in features.items():
    print(f"{name}: {feature.shape}")

# Function to create dataset with specific feature type
def create_feature_dataset(dataset, feature_type='mfcc'):
    """Create dataset with specific feature type"""
    def process_example(example):
        audio = example['audio']
        text = example['text']
        
        # Extract features based on type
        if feature_type == 'mfcc':
            features = tf.py_function(
                func=lambda a: feature_extractor.extract_mfcc(a),
                inp=[audio],
                Tout=tf.float32
            )
        elif feature_type == 'mel_spectrogram':
            features = tf.py_function(
                func=lambda a: feature_extractor.extract_mel_spectrogram(a),
                inp=[audio],
                Tout=tf.float32
            )
        elif feature_type == 'wavelet':
            features = tf.py_function(
                func=lambda a: feature_extractor.extract_wavelet_time_frequency(a),
                inp=[audio],
                Tout=tf.float32
            )
        elif feature_type == 'combined':
            features = tf.py_function(
                func=lambda a: feature_extractor.extract_combined_features(a),
                inp=[audio],
                Tout=tf.float32
            )
        else:
            raise ValueError(f"Unknown feature type: {feature_type}")
        
        # Convert text to numbers
        text_numbers = tf.py_function(
            func=lambda t: data_loader.text_to_numbers(t.numpy().decode('utf-8')),
            inp=[text],
            Tout=tf.int32
        )
        
        return features, text_numbers
    
    processed_ds = dataset.map(
        process_example,
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # Pad sequences
    def pad_data(x, y):
        x_padded = tf.keras.preprocessing.sequence.pad_sequences(
            x, padding='post', dtype='float32'
        )
        y_padded = tf.keras.preprocessing.sequence.pad_sequences(
            [y], padding='post', dtype='int32'
        )[0]
        return x_padded, y_padded
    
    padded_ds = processed_ds.map(
        lambda x, y: tf.py_function(
            func=lambda a, b: pad_data(a.numpy(), b.numpy()),
            inp=[x, y],
            Tout=[tf.float32, tf.int32]
        ),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    return padded_ds.batch(config.DATASET_CONFIG['batch_size']).prefetch(tf.data.AUTOTUNE)

# Create datasets for different feature types
print("\nCreating datasets for different feature types...")
feature_datasets = {}

for feature_type in config.FEATURE_CONFIG['feature_types']:
    print(f"Creating {feature_type} dataset...")
    
    train_ds = create_feature_dataset(datasets['train'], feature_type)
    val_ds = create_feature_dataset(datasets['validation'], feature_type)
    test_ds = create_feature_dataset(datasets['test'], feature_type)
    
    feature_datasets[feature_type] = {
        'train': train_ds,
        'val': val_ds,
        'test': test_ds
    }
    
    # Test the dataset
    for features, labels in train_ds.take(1):
        print(f"{feature_type} - Features shape: {features.shape}, Labels shape: {labels.shape}")
        break

print("\nFeature extraction completed!")
print("Available feature datasets:", list(feature_datasets.keys()))

# Save feature datasets info
feature_info = {}
for feature_type, datasets in feature_datasets.items():
    for features, labels in datasets['train'].take(1):
        feature_info[feature_type] = {
            'input_dim': features.shape[2],
            'output_dim': len(char_to_num)
        }

np.save('feature_info.npy', feature_info)
print("Feature information saved!")

print("\nNext: Train models using different feature types")